# Job Costing

Quantinuum Helios allows execution of programs with arbitrary control flow. This means each Hardware Quantum Credits (HQC) consumption per shot is non-uniform and dynamic to the circuit generated by the user program. As a consequence, the total HQC consumption is also dynamic and determined during runtime. Users are required to specify a maximum cost with each job submission. For a job submission with multiple programs, the maximum cost specification is per program. The maximum cost limits HQC consumption by terminating a job if the HQC consumed exceed this threshold. 

The Nexus python client, `qnexus`, introduces a `max_cost` kwarg to enable user input of the maximum cost. The Helios syntax checkers return a prediction for the maximum cost of a supplied Helios program, and this prediction should guide the user's choice of `max_cost`.

The max cost of a Guppy program is estimated via the python call, {py:func}`qnx.hugr.cost`.

## Dynamic Programs

A dynamic program contains multiple branches leading to varying HQC consumption across shots. The table below shows the sequence length (number of gates) and HQC consumption for a Repeat Until Success protocol. Each shot runs 10 iterations. Each iteration contains nested conditional blocks. As a result, the sequence length of the circuit is dynamic to branches selected during hardware execution. The program can generate up to 3 different circuits. The cost per circuit is calculated using Quantinuum's HQC formula.

The Guppy program is compiled to a Heirarchical Unified Graph Representation (HUGR), before upload to the Nexus database. This results in a local instance of a `HUGRRef`, a reference to the uploaded HUGR. This reference must be used to query for the maximum cost estimate from the syntax checker.

In [1]:
r"""Approximate Rz(Arctan(2)) using Repeat Until Success Protocol"""

import math

from guppylang import guppy
from guppylang.std.builtins import result, owned, comptime, panic
from guppylang.std.angles import angle
from guppylang.std.quantum import qubit, measure, t, h, z, cx, rz, discard

N = 10

@guppy
def compute_arctan() -> float:
    return comptime(math.atan(2.0)/math.pi)


@guppy
def part1(
    auxiliary: qubit @ owned,
    resource: qubit
) -> bool:
    h(auxiliary)
    t(auxiliary)
    h(resource)
    cx(resource, auxiliary)
    t(auxiliary)
    h(auxiliary)
    result1 = measure(auxiliary)
    return result1


@guppy
def part2(
    resource: qubit @ owned,
    target: qubit
) -> bool:
    h(target)
    t(target)
    z(target)
    cx(target, resource)
    t(resource)
    h(resource)
    result2 = measure(resource)
    return result2


@guppy
def part3(target: qubit @ owned) -> bool:
    arctan2 = angle(compute_arctan())
    rz(target, arctan2)
    h(target)
    result3 = measure(target)
    return result3


@guppy
def main() -> None:
    for _ in range(comptime(N)):
        auxiliary = qubit()
        resource = qubit()

        result1 = part1(auxiliary, resource)
        result("result1", result1)
        if not result1:
            target = qubit()
            result2 = part2(resource, target)
            result("result2", result2)
            if not result2:
                result3 = part3(target)
                result("result3", result3)
            else:
                discard(target)
        else:
            discard(resource)

hugr_binary = main.compile()

The HUGR binary is uploaded to the Nexus database with a name including a unique suffix.

In [7]:
import qnexus as qnx
import uuid

unique_suffix = uuid.uuid1()

project = qnx.projects.get_or_create("Helios-Samples")
qnx.context.set_active_project(project)

ref_hugr = qnx.hugr.upload(
    hugr_binary, 
    name=f"repeat-until-success-{unique_suffix}"
)

## Maximum Cost Prediction

The Helios syntax checker provides a prediction for the max cost of a Guppy program and QIR program **(Add link after QIR release this week)**. The qnexus function, {py:func}`qnexus.hugr.cost` queries the `Helios-1SC` syntax checker for the max cost prediction. The function requires the HUGR reference and the number of shots to be passed as keyword arguments.

In [10]:
prediction = qnx.hugr.cost(
    programs=[ref_hugr], 
    n_shots=[10]
)

In [11]:
print(f"Max Cost Prediction: {prediction} HQC")

Max Cost Prediction: 5.93 HQC


## Job Submission

The job submission workflow uses the {py:func}`start_execute_job` which requires the `max_cost` kwarg. If HQC consumption across all shots exceeds specified max cost, the program is terminated.

`QuantinuumConfig` accepts the maximum shot prediction for the syntax checker result, and the instance is supplied to the execute job submission function.

In [ ]:
import numpy as np

config = qnx.QuantinuumConfig(
    device_name="Helios-1E",
    max_cost=np.ceil(prediction)
)

In [ ]:
result_ref = qnx.start_execute_job(
    programs=[ref_hugr],
    n_shots=[10],
    backend_config=config,
    name=f"Helios-Emulator-{unique_suffix}"
)

In [ ]:
qnx.jobs.wait_for(result_ref)
job_result = qnx.jobs.results(result_ref)[0].download_result()

In [ ]:
print(job_result)